In [1]:
#Optimizing SVM parameters

In [2]:
#Imports
import pandas as pd
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import GridSearchCV

In [ ]:
#Load Data
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('AAL.csv')
df.head(7)

In [ ]:
#Create the lists / X and Y data set
time = []
prices = []
print(time)
print(prices)

In [ ]:
#Get the number of rows and columns in the data set
df.shape

In [ ]:
#Print the last row of data (this will be the data that we test on)
df.tail(1)

In [ ]:
#Get all of the data except for the last row
df = df.head(len(df)-1)
print(df.shape)

In [ ]:
df_dates = df.loc[:,'Date'] # Get date rows
df_open = df.loc[:,'Close'] # Get close rows

In [ ]:
# Create the independent data set X as dates
x = 0
y = 0
for moving_time in df_dates:
  if y == 0:
    time.append(int(x))
    y += 1
  else:
    x += 1
    time.append(int(x))

#Create dependent data set Y as prices
for open_price in df_open:
  prices.append(float(open_price))

In [ ]:
print(time)
print(prices)

In [ ]:
#Split data into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(time, prices, test_size = 0.5)

In [ ]:
#Optimization variables
#C
C_value = []
C_counter = 0
while C_counter < 1e3:
  C_counter += 10000
  C_value.append(C_counter)
#Degree
D_value = []
D_counter = 0
while D_counter < 10:
  D_counter += 1
  D_value.append(D_counter)
#coef0
Coef_value = []
Coef_counter = 0
while Coef_counter < 10:
  Coef_counter += 1
  Coef_value.append(Coef_counter)

In [ ]:
#Formatting
formatter = preprocessing.LabelEncoder()
#Time formatting
times = np.array(time).reshape(-1, 1)
time_variable = np.array(formatter.fit_transform(time)).reshape(-1, 1)
#Price formatting
price_variable = np.array(formatter.fit_transform(prices)).reshape(-1, 1)
print(utils.multiclass.type_of_target(price_variable.ravel()))

In [ ]:
#Optimization
SVModel = SVC()
param = {
    'kernel': ('rbf', 'linear'),
    'C': C_value,
    'degree': D_value,
    'coef0': Coef_value,
    'gamma': ('auto', 'scale')}
GridS = GridSearchCV(SVModel, param, cv=2)

In [ ]:
#Fitting Optimized Data
GridS.fit(time_variable, price_variable)

In [ ]:
#Best Parameters
Best = GridS.best_params_
Best

In [ ]:
#Predict
def predict_prices(time_variable, price_variable, x):
  
  svr_lin = SVR(kernel = Best['kernel'], C = Best['C'], coef0 = Best['coef0'], degree = Best['degree'], gamma = Best['gamma'])
  

  #Train model
  svr_lin.fit(times, prices)


  #Plot the models on a graph to see which is the best
  plt.scatter(times, prices, color = 'black', label = 'Data')

  plt.scatter(times, svr_lin.predict(times), color = 'blue', label = 'Linear')

  plt.xlabel('Date')
  plt.ylabel('Price')
  plt.title('ML Prediction')
  plt.legend()
  plt.show()
  #Returning model predictions
  return svr_lin.predict(x)[0]

In [ ]:
#Predict price of stock
predicted_price = predict_prices(time_variable, price_variable, [[250]])
print(predicted_price)